# Homework 4: Optimization



In this homework, you will act as  the logistics expert of a distribution company, that wants to figure out how to most efficiently serve its customers. This company has one sourcing location and nine customers.
The demand from each customer is equal, and has to be served through
one of the company distribution points. The goods are shipped from the sourcing location to the
distribution point and then transported to the individual customers. There are three possible candidate
distribution point locations to choose from. You are given a table of data that shows the cost of transportation from each
of the distribution points to each customer. These transportation costs are directly
proportional to the distance between the distribution point and the customer locations. The company
wants to establish a distribution point at two of these three locations in order to minimize the maximum
distance from each customer to the nearest distribution point, so that no one customer is particularly far from their distribution location. What is the best way to do this?


First, let's read in our data, which shows the costs for shipping to each customer from each distribution point location. **Run the code below to load the `.csv` file into a `Pandas` data frame. 

In [22]:
import pandas as pd
shipping_data = pd.read_csv('customer_distribution_distances.csv')
shipping_data.head()

,Customer,Location A,Location B,Location C
0,1,170,170,50
1,2,200,150,50
2,3,170,80,70
3,4,100,170,70
4,5,120,60,60


Now, let's put together our optimization problem piece by piece. This problem is requiring us to make binary decisions, so it is an integer program. The first step is figuring out our decision variables, and an objective to minimize or maximize. In this case, the company wants to minimize the maximum distance from each customer to their distribution point. 

Let's define our set of customers as $C$ and our set of distribution points as $D$. If we write the distance from customer $i$ to distribution point $j$ as $d_{ij}$, and define a binary variable $x_{ij}$ to be 1 if we supply customer $i$ from distribution point $j$ and 0 otherwise, we can write the objective function as $\max\{d_{ij}x_{ij} \: \forall \:  i \in C, j \in D  \}$ - in other words, find the largest $d_{ij}x_{ij}$ over all combinations of $i$ and $j$. This is the value we want to minimize.

However, recall from lecture that the $\max$ function is a nonlinear function. In order to perform our optimization efficiently, we want to linearize this. In lecture, we saw that we can define another variable (let's call it $T$) to be this maximum  $d_{ij}x_{ij}$ - then we simply need to add constraints such that  $T$ is greater than all $d_{ij}x_{ij}$ values. Since $T$ is being minimized, $T$ will necessarily take on the largest $d_{ij}x_{ij}$ value. Do you see why? 

Let's start by initializing our model, our sets $D$ and $C$, and declare our variables, just as we did in lecture. We will also want a variable to describe which of our distribution points we use - remember this is the main decision the company is interested in! Let's call it $y_j$, one for each candidate distribution point $j$, such that it is 1 if we use that distribution point, and 0 if we don't. **Run the cell below to define our indices and variables**. Note that while $x_{ij}$ and $y_j$ are both binary, we let $T$ take on any non-negative real value. 

In [23]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

#distribution indices
D = range(3)
#customer indices 
C = range(9)


# declare decision variables
model.y = Var(D,domain=Binary)
model.x = Var(C,D,domain=Binary)
model.T = Var(domain=NonNegativeReals)

Now, let's define our objective function. We are just trying to minimize $T$, so our expression is very simple, it is just $T$ - this is a linear function, since $T$ is a variable! The objective function below is set up as a maximization problem. **Change this to be a minimization problem, then run the cell below.**

In [24]:
# declare objective
# model.min_max_cost = Objective(expr =  model.T, sense=maximize)
model.min_max_cost = Objective(expr =  model.T, sense=minimize)

Now we need our constraints that ensure $T$ is equivalent to the maximum distance from a distribution point to a customer. First we need our distance data in an easily useable form. **Run the cell below to convert the data from a `Pandas` data frame to a `Numpy` matrix.**

In [25]:
import numpy as np
d = np.array(shipping_data.iloc[0:9,1:4])
print(d)

[[170 170  50]
 [200 150  50]
 [170  80  70]
 [100 170  70]
 [120  60  60]
 [ 40 150 150]
 [ 80 160 120]
 [100 140  40]
 [100  40  40]]


We can now add our constraints to ensure that $T$ is a maximum. Since there will be a lot of constraints, the cell below uses a `ConstraintList`, just as we saw in lecture. For our contraints, we need $T \geq d_{ij} x_{ij}, \forall \: i \in C, j \in D$. **Run the cell below to create the constraint list, then add this constraint for each customer $i$ and distribution point $j$.** 

In [26]:
# add model constraints
model.constraints = ConstraintList()
#maximum distance linearization
for i in C:
  for j in D: 
    model.constraints.add(model.T >= d[i,j]*model.x[i,j])

We also need a constraint that ensures that 2 (and only 2) distribution points are open. We can do this by ensuring that $y_1 + y_2 + y_3 = 2$, or, with summation notation, $\sum_j y_j = 2$. Do you see why? The code in the cell below ensures that only one distribution point is open. **Change this so that 2 distribution points are open, then run the cell**.

In [27]:
# only two warehouses
# model.constraints.add(sum(model.y[j] for j in D) == 1)
model.constraints.add(sum(model.y[j] for j in D) == 2)

We also need constraints that make sure that each customer demand is supplied by one and only one supply center (less than one means that the customer is not satisfied, and more than one means that we are unnessarily oversupplying the customer at our expense). We need one of these constraints for each customer, so we'll again use a loop, now over each customer $i$, and for each customer sum up all $x_{ij}$ associated with this customer, and make sure that sum equals one. In math, this is $\sum_{j \in D} x_{ij} = 1$ for each customer $i$. The code below ensures that each customer will be supplied by 2 distribution points. **Change the code so that each customer is supplied by one and only one distribution point.**

In [28]:
# customer demand satisfied by one and only one distribution center
for i in C:
   # model.constraints.add(sum(model.x[i,j] for j in D) == 2)
   model.constraints.add(sum(model.x[i,j] for j in D) == 1)

Finally, we need a constraint that ensures that customers are only supplied by distibution points that are open - in other words, if $x_{ij}$ is one, $y_j$ with the same $j$ must also be 1. In lecture, we achieved this with what we called a forcing constraint. If $x_{ij} \leq y_j$ for all $i$ and $j$, then we have ensured that, if any $x_{ij}$ for any customer $i$ is one, than $y_j$ for that $j$ is also 1. **The code below writes these constraints in a loop, but gets the inequality wrong. Fix the inequality to be correct, then run the cell to define the constraints.**

In [29]:
#forcing constraints (distribution cannot supply unless open)
for i in C:
  for j in D:
    # model.constraints.add(model.x[i,j] >= model.y[j])
    model.constraints.add(model.x[i,j] <= model.y[j])


We now have defined our whole model, which summarized mathematically is as follows: 

$$
\begin{aligned}
& \underset{T,x,y}{\text{minimize}}
& & T\\
& \text{subject to}
& & T \geq d_{ij} x_{ij}, \forall \: i \in C, j \in D \\
& & & \sum_{j \in D} y_{j} = 2 \\
& & & \sum_{j \in D} x_{ij} = 1, \forall \:  i \in C \\
& & & x_{ij} \leq y_j, \forall \: i \in C, j \in D \\
&&& x_{ij}, y_j \in \{0,1\} , \forall \: i \in C, j \in D \\
&&& T \geq 0
\end{aligned}
$$




 **Run the cell below to solve the model with the `glpk` solver.** 

In [31]:
# SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()
# Download and install glpsol from https://sourceforge.net/projects/winglpk/
SolverFactory('glpk', executable='glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 80.0
  Upper bound: 80.0
  Number of objectives: 1
  Number of constraints: 65
  Number of variables: 32
  Number of nonzeros: 139
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 19
      Number of created subproblems: 19
  Error rc: 0
  Time: 0.05406332015991211
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

Let's view the optimal minimized maximum distance, open distribution points and which distribution points supply which customers at optimality. Since it is convenient to view these variables in tables with their associated customer and distribution point lables, we create `Pandas` Data Frames for both `model.x` and `model.y`, just as we did in lection. .

In particular, to look at the values of `model.x`, we borrow the structure of the `shipping_data` table (which has labels for both customers and distribution points) using the `.copy()` method, then weloop through all indicies of `model.x` and insert these flow values in their corresponding spot in the new Data Frame `x`. **Run the cell below to print out the optimal cost and decisions for this model, examine the output, create a new text cell, and briefly explain in that cell your recommendations to the company.**

In [32]:
# display solution
print('\nProfit = ', model.min_max_cost())

print('\nDecision Variables')

print('\ny: which distribution centers open?')
#print y
print(pd.DataFrame({"Distribution Center":['A','B','C'],"Open?":[model.y[i].value for i in D]}))

#print x
print('\nx: flow from warehouse in city i to region j')
x = shipping_data.copy()
for i in C:
  for j in D:
    x.iloc[i,j+1] = model.x[i,j].value
print(x)



Profit =  80.0

Decision Variables

y: which distribution centers open?
  Distribution Center  Open?
0                   A    1.0
1                   B    0.0
2                   C    1.0

x: flow from warehouse in city i to region j
   Customer  Location A  Location B  Location C
0         1           0           0           1
1         2           0           0           1
2         3           0           0           1
3         4           0           0           1
4         5           0           0           1
5         6           1           0           0
6         7           1           0           0
7         8           0           0           1
8         9           0           0           1


## Recommendations to the Company

The two recommended distribution points are `Location A`  and `Location C`. `Location B` is not recommended.
- `Location A` can serve customers `6` and `7`
- `Location B` caan serve customers `1`, `2`, `3`, `4`, `5` and `8`, `9`.